In [2]:

import os
from dotenv import load_dotenv
_ = load_dotenv()

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import AzureChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

In [3]:
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

'5EfmGxDSlWg9UtLW4FsVqqUsE8dDqS6VfPdxbtaJYZ3jmJyFj6GVJQQJ99BEACfhMk5XJ3w3AAAAACOGQ1xl'

In [6]:
# Azure AI Foundry configuration
model = AzureChatOpenAI(
    azure_endpoint=endpoint,
    api_key=azure_openai_key,
    api_version="2024-12-01-preview",
    azure_deployment="gpt-4.1",  # Your deployment name
    model="gpt-4.1",
    temperature=0
)

In [12]:
model.invoke([SystemMessage(content="Ready for working?")]).content

"Yes, I'm ready to help! What do you need assistance with today?"

## Import Prompts

In [ ]:
import sys
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Load the intent classifier prompt from the text file
with open(os.path.join(parent_dir, "prompts", "intent_classifier.txt"), "r") as f:
    intent_classifier_prompt = f.read()

intent_classifier_prompt

'You are an intent classifier for the ChatSupervisor agent, the central brain of a worker assistant. Your primary responsibility is to analyze the user\'s latest message in the context of the chat history and determine the user\'s primary intent.\n\nYour response must be only the name of the single, most appropriate agent or tool to handle this intent. Do not provide any other text, explanation, or preamble. Respond with the agent name on a single line, with no extra whitespace, punctuation, or formatting.\n\nThese are your only valid routing options:\n\n1.  InformationRetrievalAgent\n    - Purpose: To answer generic questions about work policies, procedures, or "how-to" guides using the internal knowledge base.\n    - Examples:\n        - "How do I call in sick?"\n        - "What are the company rules for overtime?"\n        - "How are reservations calculated?"\n\n2.  ActionExecutionAgent\n    - Purpose: To handle personalized requests that require fetching the user\'s own specific da

## Agent Set-up

In [ ]:
# Agent State
class AgentState(TypedDict):
   messages: Annotated[list[AnyMessage], "Conversation messages"]
   next_action: Annotated[str, "Next action to take"]
   retrieved_data: Annotated[str, "Data retrieved from tools"]
   error_message: Annotated[str, "Error message if any"]
   attempts: Annotated[int, "Number of attempts made"]
   max_retries: Annotated[int, "Maximum number of retries allowed"]

In [ ]:
class ChatSupervisorAgent():
    def __init__(self, model, tools, system="", checkpointer=None):
        self.model = model
        self.system = system
        self.tools = {tool.name: tool for tool in tools}

        graph = StateGraph(AgentState)
        graph.add_node("next_action:", self.next_action)
        graph.add_conditional_edges(
            "next_action",
            self.next_action,
            {
                "action_exection": "action_exection",
                "information_retrieval": "information_retrieval",
                "other": "fallback_tool",
            }
            )

        def next_action(self, state: AgentState) -> str:
            '''Logic to determine the next action based on the current state'''
            last_message = state["messages"][-1]
            